In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
import glob, os, sys
import pickle

In [10]:
from keras import backend as K
from keras.preprocessing import image
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers

from sklearn.model_selection import train_test_split

from keras.applications.resnet50 import ResNet50 as resnet
from keras.applications.resnet50 import preprocess_input as resnet_pp

from keras.optimizers import Adam

In [ ]:
main_path = r'./POETdataset/PascalImages/'
classes = ['aeroplane', 'boat', 'dog', 'bicycle', 'cat', 'cow', 'diningtable', 'horse', 'motorbike','sofa']
IMG_SIZE = 224

In [3]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
def load_object(filename):
    with open(filename, 'rb') as fp:
        return pickle.load(fp)

In [4]:
# Build a model by adding preprocessing before the pretrained CNN
def get_feature_extraction_model():
    cnn_object, pp_function, img_size = _get_pretrained_model()
    model = keras.models.Sequential()
    cnn_model = cnn_object(weights='imagenet', include_top=False, pooling='max')
    model.add(keras.layers.Lambda(pp_function, name='preprocessing', input_shape=(img_size, img_size, 3)))
    model.add(cnn_model)
    return model

# Unpacking information from the models dictionary
def _get_pretrained_model():
    cnn_object = resnet
    pp_function = resnet_pp
    img_size = IMG_SIZE
    return cnn_object, pp_function, img_size

def get_features(files, model):
    # Load images based on the size of the Lambda layer 
    # provided as the first layer before the pretrained CNN
    x = np.array([image.img_to_array(image.load_img(file, target_size=(model.layers[0].input_shape[1], model.layers[0].input_shape[1]))) for file in files])
    return model.predict(x, verbose=1)

In [5]:
model = get_feature_extraction_model()

In [15]:
classes = [word + '*' for word in classes]
files_list = [glob.glob(main_path + class_) for class_ in classes]

for files in files_list:
    assert len(files) > 0
    
new_dir = './features'
if not os.path.exists(new_dir):
    os.makedirs(new_dir)

files_dict = {class_name.replace('*', ''): class_files for class_name, class_files in zip(classes, files_list)}
for class_number, (class_name, files) in enumerate(files_dict.items()):
    print(class_name)
    features = get_features(files, model)
    print(features.shape)
    save_object(features, new_dir + os.path.sep + class_name+'_features.pkl')

aeroplane
666/666 [==============================] - 3s 4ms/step
(666, 2048)
boat
504/504 [==============================] - 2s 4ms/step
(504, 2048)
dog
1257/1257 [==============================] - 5s 4ms/step
(1257, 2048)
bicycle
536/536 [==============================] - 2s 4ms/step
(536, 2048)
cat
1051/1051 [==============================] - 4s 4ms/step
(1051, 2048)
cow
301/301 [==============================] - 1s 4ms/step
(301, 2048)
diningtable
498/498 [==============================] - 2s 4ms/step
(498, 2048)
horse
480/480 [==============================] - 2s 4ms/step
(480, 2048)
motorbike
510/510 [==============================] - 2s 4ms/step
(510, 2048)
sofa
467/467 [==============================] - 2s 4ms/step
(467, 2048)


In [17]:
X = []
y = []
for class_number, (class_name, files) in enumerate(files_dict.items()):
    features = load_object(new_dir + os.path.sep + class_name + '_features.pkl').reshape(-1,2048)
    print(features.shape)
    X.append(features)
    y.append([class_number] * features.shape[0])
    
X = np.concatenate(X)
y = np.concatenate(y)
y = keras.utils.to_categorical(y)
X.shape, y.shape

(666, 2048)
(504, 2048)
(1257, 2048)
(536, 2048)
(1051, 2048)
(301, 2048)
(498, 2048)
(480, 2048)
(510, 2048)
(467, 2048)


((6270, 2048), (6270, 10))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5643, 2048), (5643, 10), (627, 2048), (627, 10))

In [19]:
dense_model = Sequential()
# model.add(Flatten(input_shape=(1,1,2048)))
dense_model.add(Dense(units=128, activation='relu', input_shape=(2048,)))
dense_model.add(Dropout(0.25))
dense_model.add(Dense(units=10, activation='softmax'))

dense_model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy'])
history = dense_model.fit(X_train,y_train,
                    batch_size = 32,
                    epochs = 100,
                    validation_data=(X_test, y_test))

Train on 5643 samples, validate on 627 samples
Epoch 1/100
5643/5643 [==============================] - 1s 207us/step - loss: 2.0218 - acc: 0.6309 - val_loss: 0.6444 - val_acc: 0.8054
Epoch 2/100
5643/5643 [==============================] - 0s 85us/step - loss: 0.7804 - acc: 0.7510 - val_loss: 0.5533 - val_acc: 0.8453
Epoch 3/100
5643/5643 [==============================] - 0s 82us/step - loss: 0.6301 - acc: 0.7920 - val_loss: 0.5360 - val_acc: 0.8293
Epoch 4/100
5643/5643 [==============================] - 0s 87us/step - loss: 0.5957 - acc: 0.8042 - val_loss: 0.4803 - val_acc: 0.8453
Epoch 5/100
5643/5643 [==============================] - 0s 83us/step - loss: 0.5334 - acc: 0.8111 - val_loss: 0.5447 - val_acc: 0.8421
Epoch 6/100
5643/5643 [==============================] - 0s 82us/step - loss: 0.4940 - acc: 0.8304 - val_loss: 0.4923 - val_acc: 0.8581
Epoch 7/100
5643/5643 [==============================] - 0s 85us/step - loss: 0.4545 - acc: 0.8467 - val_loss: 0.5963 - val_acc: 0.8325


5643/5643 [==============================] - 0s 85us/step - loss: 0.1728 - acc: 0.9351 - val_loss: 1.0923 - val_acc: 0.8150
Epoch 61/100
5643/5643 [==============================] - 0s 85us/step - loss: 0.2072 - acc: 0.9238 - val_loss: 1.1507 - val_acc: 0.8166
Epoch 62/100
5643/5643 [==============================] - 0s 82us/step - loss: 0.1857 - acc: 0.9321 - val_loss: 1.0152 - val_acc: 0.8405
Epoch 63/100
5643/5643 [==============================] - 0s 82us/step - loss: 0.1609 - acc: 0.9335 - val_loss: 1.1461 - val_acc: 0.8341
Epoch 64/100
5643/5643 [==============================] - 0s 83us/step - loss: 0.1730 - acc: 0.9327 - val_loss: 0.9475 - val_acc: 0.8246
Epoch 65/100
5643/5643 [==============================] - 0s 88us/step - loss: 0.1719 - acc: 0.9307 - val_loss: 1.1590 - val_acc: 0.8325
Epoch 66/100
5643/5643 [==============================] - 0s 86us/step - loss: 0.1402 - acc: 0.9428 - val_loss: 1.2800 - val_acc: 0.8246
Epoch 67/100
5643/5643 [==============================